In [1]:
from src.utils import *
import numpy as np
from sklearn.cluster import KMeans

Can't initialize NVML


In [2]:
config = {"dataset": "sst2", "topics": "lda"}
data_train, data_val, data_test = load_data(config)
x = [data_test[i]['sentence'] for i in range(100)]
# x = [xi for xi in x if len(xi.split()) > 1]
# topics, word2idx = get_topics(config, x)

Found cached dataset glue (/home/steinad/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/steinad/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/steinad/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
topics, word2idx = get_topics({"dataset": "sst2", "topics": "lda"}, x)
roberta_shap_vals = load("shap_vals_gpt2_sst2")
roberta_topic_vals = load("topic_vals_distilroberta_sst2_lda").reshape(-1, 31)
roberta_word_vals = load("word_vals_distilroberta_sst2_lda")
gpt2_topic_vals = load("topic_vals_gpt2_sst2_lda").reshape(-1, 31)
gpt2_word_vals = load("word_vals_gpt2_sst2_lda")

In [4]:
base_path = "../scripts/"
topics_matrix_df = pd.read_csv(base_path + ("../data/processed_LDA_files/" + "sst2" + ".csv"))
word2idx = dict(zip(topics_matrix_df["words"], range(len(topics_matrix_df["words"]))))
topics_matrix_df.drop(columns=["words"], inplace=True)
topics_matrix_df = topics_matrix_df.T
topics_raw = topics_matrix_df.to_numpy()
sorted_words = np.argsort(topics_raw, axis=1)
idx2word = {i: word for word, i in word2idx.items()}
for i in range(topics_raw.shape[0]):
    print(i, [idx2word[word_idx] for word_idx in sorted_words[i][::-1][:10]])

0 ['dull', 'nor', 'neither', 'fresh', 'far', 'every', 'original', 'goes', 'day', 'watch']
1 ['action', 'wo', 'when', 'still', 'away', 'compelling', 'theater', 'tale', 'worthwhile', 'imagination']
2 ['plot', 'picture', 'material', 'rather', 'boring', 'brilliant', 'mess', 'high', 'obvious', 'tired']
3 ['acting', 'direction', 'clever', 'script', 'dialogue', 'long', 'poorly', 'music', 'screenplay', 'journey']
4 ['energy', 'intelligence', 'cinematic', 'fascinating', 'entertaining', 'two', 'documentary', 'impressive', 'passion', 'touch']
5 ['minutes', 'long', 'first', 'many', 'nearly', 'nothing', 'short', 'care', 'feels', 'watch']
6 ['bland', 'without', 'dumb', 'american', 'thoroughly', 'plain', 'him', 'beauty', 'cute', 'deserves']
7 ['better', 'worst', 'year', 'could', 'films', 'look', 'him', 'those', 'ugly', 'hollywood']
8 ['performances', 'cast', 'full', 'performance', 'often', 'really', 'solid', 'them', 'strong', 'while']
9 ['visual', 'own', 'beautiful', 'rich', 'kind', 'spirit', 'cast',

In [5]:
idx = 4
print(roberta_word_vals[idx])
print(roberta_topic_vals[idx])
print(sum(roberta_word_vals[idx]))
print(sum(roberta_topic_vals[idx]))
print(x[idx])

[0.01852864772081375, 0.0006963387131690979, 0.005582142621278763, 0.022627518512308598, -0.016558256931602955, -0.010342827066779137, 0.0244749765843153, 0.0010270774364471436, 0.007224409756335345, 0.0046737016263333235, -0.0031853812662037935, -0.009010147388008509, 0.07451375192877921, 0.03656286365267905, -0.00515490033748475, -0.12643968813283596, -0.11865620104292633, 0.13449292054936063, 0.03400407629934224]
[-0.01233354  0.         -0.00716496 -0.00073731 -0.00252903 -0.01992763
 -0.01108731 -0.00053977  0.          0.         -0.00170561  0.02390985
 -0.00040094 -0.0164027  -0.00584376 -0.00891486  0.00028088  0.00095957
 -0.00568907 -0.00026043 -0.00095403 -0.0260686  -0.00169033 -0.00527572
 -0.01523971  0.00248917  0.00110983  0.          0.         -0.00030594
  0.18938297]
0.075061023235321
0.07506102323532102
lathan and diggs have considerable personal charm , and their screen rapport makes the old story seem new .


In [31]:
X = roberta_topic_vals / np.sum(roberta_topic_vals, axis=1, keepdims=True)
kmeans = KMeans(n_clusters=10).fit(X)
clabels = kmeans.labels_

diffs = np.zeros(10)
for c in range(10):
    diffs[c] = np.sum(np.abs(np.sum(roberta_topic_vals[clabels == c], axis=0) - np.sum(gpt2_topic_vals[clabels == c], axis=0)) / np.sum(clabels == c))
most_diff_cluster = np.argsort(diffs)[::-1][0]
print(diffs)
print(most_diff_cluster)
diff = np.abs(np.sum(roberta_topic_vals[clabels == most_diff_cluster], axis=0) - np.sum(gpt2_topic_vals[clabels == most_diff_cluster], axis=0)) / np.sum(clabels == most_diff_cluster)
print(np.argsort(diff)[::-1][:3])

for idx in np.nonzero(clabels == most_diff_cluster)[0]:
    print(idx, x[idx])

[0.35931513 0.74429793 1.18468939 0.46055958 0.3257553  1.45500909
 1.02774093 1.00907149 1.04191695 0.51910663]
5
[30 25 19]
29 here 's a british flick gleefully unconcerned with plausibility , yet just as determined to entertain you .


In [47]:
diffs = np.sum(np.abs(roberta_topic_vals - gpt2_topic_vals), axis=1)
biggest_diff = np.argsort(diffs)
print(diffs[biggest_diff[-5:]])

[1.18468939 1.24345609 1.34729508 1.45500909 1.50728886]


In [32]:
ind = 29 #biggest_diff[-1]
print(x[ind])
topic_names = np.array([f"topic_{i}" for i in range(36)])
topic_values, sorted_names = sort_shap(roberta_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

print()
topic_values, sorted_names = sort_shap(gpt2_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

here 's a british flick gleefully unconcerned with plausibility , yet just as determined to entertain you .
topic_19: -0.1553845656310764
topic_30: -0.10612585631315596
topic_25: -0.0935087014799168
topic_14: -0.03646532557338526
topic_15: -0.01593457268374577
topic_22: 0.02807360821965514
topic_1: 0.045695628701287475
topic_21: 0.0492626937546679
topic_26: 0.09268165987427164
topic_10: 0.22321857630317818
0.06044590473175046

topic_19: -0.3872873015917917
topic_0: 0.0
topic_17: 0.0
topic_16: 0.0
topic_13: 0.0
topic_15: 0.029227123956237402
topic_4: 0.03711597906478719
topic_26: 0.05891144657342237
topic_25: 0.14575828416624173
topic_30: 0.3104190980375279
0.358201801776886


In [49]:
roberta_feat_imp = np.sum(np.abs(roberta_topic_vals), axis=0)
gpt2_feat_imp = np.sum(np.abs(gpt2_topic_vals), axis=0)

topic_values, sorted_names = sort_shap(roberta_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

print()
topic_values, sorted_names = sort_shap(gpt2_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

topic_18: 1.1927641473527824
topic_28: 1.2792578255936047
topic_10: 1.4127325231550543
topic_13: 1.6853790241062112
topic_12: 1.6970660445412045
topic_7: 2.672635190010829
topic_25: 2.951807518542655
topic_21: 2.9736445350846377
topic_15: 2.9963853319057185
topic_30: 24.913793117689515

topic_18: 0.8932053637634415
topic_10: 0.9632376007580045
topic_12: 1.4546492131042734
topic_23: 1.477474603109051
topic_3: 1.5033561471434458
topic_27: 2.491718571442825
topic_16: 2.4973391853080935
topic_29: 2.5416982269866084
topic_15: 2.722013871904929
topic_30: 19.318108668372073
